In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from os import path

data_dir = 'data/bank-transactions'
data_file = 'bank-transactions.csv'
data_path = path.join(data_dir, data_file)

df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [3]:
df.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046198e+06,1.048567e+06,1.048567e+06
mean,1.154035e+05,1.570875e+05,1.574335e+03
std,8.464854e+05,5.126185e+04,6.574743e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.721760e+03,1.240300e+05,1.610000e+02
50%,1.679218e+04,1.642260e+05,4.590300e+02
75%,5.765736e+04,2.000100e+05,1.200000e+03
max,1.150355e+08,2.359590e+05,1.560035e+06


In [4]:
df['CustLocation'].value_counts()

MUMBAI                         103595
NEW DELHI                       84928
BANGALORE                       81555
GURGAON                         73818
DELHI                           71019
                                ...  
ROAD (E) THANE                      1
TO APOLO PHARMACY BANGALORE         1
PILLAR NO 725 NEW DELHI             1
COMPANY BANGALORE                   1
IMPERIA THANE WEST                  1
Name: CustLocation, Length: 9355, dtype: int64

#### Droppando nulos

In [5]:
df.dropna(inplace=True)

#### Entendendo atributos

In [6]:
df.head(5)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [7]:
print(df['CustomerDOB'].values)
print(np.unique(df['CustGender'].values, return_counts=True))

['10/1/94' '4/4/57' '26/11/96' ... '18/5/89' '30/8/78' '5/3/84']
(array(['F', 'M', 'T'], dtype=object), array([280635, 760978,      1]))


In [8]:
print(np.unique(df['CustLocation'].values).shape)

(9275,)


#### Transformando atributos

Transformando Date Of Birth (DOB) em idade

In [9]:
def takeYear(dob):
    return list( map(lambda x: int(x), dob.split('/')) )[-1]

def takeMonth(dob):
    return list( map(lambda x: int(x), dob.split('/')) )[1]

def takeDay(dob):
    return list( map(lambda x: int(x), dob.split('/')) )[0]
    
df['YearDOB'] = df['CustomerDOB'].apply(takeYear)
df['MonthDOB'] = df['CustomerDOB'].apply(takeMonth)
df['DayDOB'] = df['CustomerDOB'].apply(takeDay)

df[['DayDOB', 'MonthDOB', 'YearDOB']]

,DayDOB,MonthDOB,YearDOB
0,10,1,94
1,4,4,57
2,26,11,96
3,14,9,73
4,24,3,88
...,...,...,...
1048562,8,4,90
1048563,20,2,92
1048564,18,5,89
1048565,30,8,78


Validando dias, meses e anos

In [10]:
day_valid = (df['DayDOB'] <= 31) & (df['DayDOB'] >= 1)
month_valid = (df['MonthDOB'] <= 12) & (df['MonthDOB'] >= 1)
year_valid = (df['YearDOB'] <= 99) & (df['YearDOB'] >= 0)

day_valid.all(), month_valid.all(), year_valid.all()

(True, True, False)

Removendo as entradas com valores inválidos para ano

In [11]:
df = df[year_valid]
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),YearDOB,MonthDOB,DayDOB
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0,94,1,10
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0,57,4,4
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0,96,11,26
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0,73,9,14
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5,88,3,24
...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0,90,4,8
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0,92,2,20
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0,89,5,18
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0,78,8,30


Utilizando apenas o ano para decidir idade

In [12]:
df['CustomerAge'] = 2022 - df['YearDOB'].apply(lambda x: 1900 + x if x > 22 else 2000 + x)
df[['CustomerAge', 'YearDOB']]

,CustomerAge,YearDOB
0,28,94
1,65,57
2,26,96
3,49,73
4,34,88
...,...,...
1048562,32,90
1048563,30,92
1048564,33,89
1048565,44,78


Revendo dataset

In [13]:
df.head(5)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),YearDOB,MonthDOB,DayDOB,CustomerAge
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0,94,1,10,28
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0,57,4,4,65
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0,96,11,26,26
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0,73,9,14,49
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5,88,3,24,34


In [14]:
df.drop(columns=['TransactionID', 'CustomerID', 'CustomerDOB', 'YearDOB', 'MonthDOB', 'DayDOB'], inplace=True)

In [15]:
df.head(5)

,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0,28
1,M,JHAJJAR,2270.69,2/8/16,141858,27999.0,65
2,F,MUMBAI,17874.44,2/8/16,142712,459.0,26
3,F,MUMBAI,866503.21,2/8/16,142714,2060.0,49
4,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5,34


#### Analisando `CustLocation`

In [16]:
df['CustLocation'].value_counts()[:20]

MUMBAI         97679
BANGALORE      77751
NEW DELHI      76380
GURGAON        70518
DELHI          67335
NOIDA          31675
CHENNAI        27500
PUNE           24493
HYDERABAD      21819
THANE          21006
KOLKATA        19033
GHAZIABAD      14624
NAVI MUMBAI    11932
AHMEDABAD      11266
FARIDABAD      10641
JAIPUR          9231
CHANDIGARH      8801
LUCKNOW         7338
MOHALI          5828
SURAT           5173
Name: CustLocation, dtype: int64

É possível transformar em um atributo numérico ordinal, preservando a relação de ordem pela quantidade de pessoas em cada cidade. É possível simplesmente droppar esse atributo também...

Acho que, para um primeiro teste, vamos apenas droppá-lo.

In [17]:
df.drop(columns='CustLocation', inplace=True)
df.head(5)

,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,F,17819.05,2/8/16,143207,25.0,28
1,M,2270.69,2/8/16,141858,27999.0,65
2,F,17874.44,2/8/16,142712,459.0,26
3,F,866503.21,2/8/16,142714,2060.0,49
4,F,6714.43,2/8/16,181156,1762.5,34


#### Lidando com CustGender

In [18]:
df['CustGender'].value_counts()

M    712454
F    272868
Name: CustGender, dtype: int64

In [19]:
df['CustGender'] = df['CustGender'].apply(lambda x: float(x == 'F'))
df.head(5)

,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,1.0,17819.05,2/8/16,143207,25.0,28
1,0.0,2270.69,2/8/16,141858,27999.0,65
2,1.0,17874.44,2/8/16,142712,459.0,26
3,1.0,866503.21,2/8/16,142714,2060.0,49
4,1.0,6714.43,2/8/16,181156,1762.5,34


#### Lidando com `TransactionDate`

Talvez possamos simplesmente criar três atributos, um para o dia da transação, um para o mês e outro para o ano.

In [20]:
df[['TransactionDay', 'TransactionMonth', 'TransactionYear']] = df['TransactionDate'].str.split('/', expand=True)
df.head(5)

,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge,TransactionDay,TransactionMonth,TransactionYear
0,1.0,17819.05,2/8/16,143207,25.0,28,2,8,16
1,0.0,2270.69,2/8/16,141858,27999.0,65,2,8,16
2,1.0,17874.44,2/8/16,142712,459.0,26,2,8,16
3,1.0,866503.21,2/8/16,142714,2060.0,49,2,8,16
4,1.0,6714.43,2/8/16,181156,1762.5,34,2,8,16


Validando entradas para esses três atributos:

In [21]:
df['TransactionDay'] = df['TransactionDay'].astype('float64')
df['TransactionMonth'] = df['TransactionMonth'].astype('float64')
df['TransactionYear'] = df['TransactionYear'].astype('float64')

In [22]:
transday_valid = (df['TransactionDay'] <= 31) & (df['TransactionDay'] >= 1)
transmonth_valid = (df['TransactionMonth'] <= 12) & (df['TransactionMonth'] >= 1)
transyear_valid = (df['TransactionYear'] <= 99) & (df['TransactionYear'] >= 0)

transday_valid.all(), transmonth_valid.all(), transyear_valid.all()

(True, True, True)

Tudo certo. Então, é só droppar o atributo antigo.

In [23]:
df.drop(columns='TransactionDate', inplace=True)
df.head(5)

,CustGender,CustAccountBalance,TransactionTime,TransactionAmount (INR),CustomerAge,TransactionDay,TransactionMonth,TransactionYear
0,1.0,17819.05,143207,25.0,28,2.0,8.0,16.0
1,0.0,2270.69,141858,27999.0,65,2.0,8.0,16.0
2,1.0,17874.44,142712,459.0,26,2.0,8.0,16.0
3,1.0,866503.21,142714,2060.0,49,2.0,8.0,16.0
4,1.0,6714.43,181156,1762.5,34,2.0,8.0,16.0


#### Garotice: transformar os horários de transação em UNIX timestamp

É possível condensar `TransactionDay`, `TransactionMonth`, `TransactionYear` e `TransactionTime` em um único atributo: `TransactionTimestamp`. Vamo tentar fazer essa garotice.

Primeiro, bora descobrir uma coisa: quão grande são os valores em `TransactionTime`?

In [24]:
aux = df['TransactionTime'].apply(lambda x: len(str(x)))
(aux == 7).any(), (aux == 6).any(), (aux == 5).any(), (aux == 4).any(), \
                  (aux == 3).any(), (aux == 2).any(), (aux == 1).any()

(False, True, True, True, True, True, True)

Oi? Tem entrada com tamanho 1 nesse atributo? Bora ver isso aí:

In [25]:
idx = (aux == 1).idxmax()
df.loc[idx]

CustGender                      1.00
CustAccountBalance         125336.73
TransactionTime                 4.00
TransactionAmount (INR)        50.00
CustomerAge                    39.00
TransactionDay                 26.00
TransactionMonth                9.00
TransactionYear                16.00
Name: 35289, dtype: float64

Realmente, tem mesmo... F...

Vamos fazer essa separação algebricamente, então:

In [26]:
df['TimeHours']   = df['TransactionTime'] // 10000
df['TimeMinutes'] = (df['TransactionTime'] // 100) - ((df['TransactionTime'] // 10000) * 100)
df['TimeSeconds'] = df['TransactionTime'] - (df['TransactionTime'] // 100) * 100

df.head(5)

,CustGender,CustAccountBalance,TransactionTime,TransactionAmount (INR),CustomerAge,TransactionDay,TransactionMonth,TransactionYear,TimeHours,TimeMinutes,TimeSeconds
0,1.0,17819.05,143207,25.0,28,2.0,8.0,16.0,14,32,7
1,0.0,2270.69,141858,27999.0,65,2.0,8.0,16.0,14,18,58
2,1.0,17874.44,142712,459.0,26,2.0,8.0,16.0,14,27,12
3,1.0,866503.21,142714,2060.0,49,2.0,8.0,16.0,14,27,14
4,1.0,6714.43,181156,1762.5,34,2.0,8.0,16.0,18,11,56


Passando tudo pra inteiro por causa do timestamp

In [27]:
def createTimestamp(df):
    import datetime
    
    times = np.array(df.values[:, -6:], dtype='int64')
    timestamps = []
    
    for time in times:
        aux = datetime.datetime(2000 + time[2], time[1], time[0], time[3], time[4], time[5])
        timestamps.append(aux.timestamp())
    
    return np.array(timestamps)

df['TransactionTimestamp'] = createTimestamp(df)

In [28]:
df.head(5)

,CustGender,CustAccountBalance,TransactionTime,TransactionAmount (INR),CustomerAge,TransactionDay,TransactionMonth,TransactionYear,TimeHours,TimeMinutes,TimeSeconds,TransactionTimestamp
0,1.0,17819.05,143207,25.0,28,2.0,8.0,16.0,14,32,7,1.470159e+09
1,0.0,2270.69,141858,27999.0,65,2.0,8.0,16.0,14,18,58,1.470158e+09
2,1.0,17874.44,142712,459.0,26,2.0,8.0,16.0,14,27,12,1.470159e+09
3,1.0,866503.21,142714,2060.0,49,2.0,8.0,16.0,14,27,14,1.470159e+09
4,1.0,6714.43,181156,1762.5,34,2.0,8.0,16.0,18,11,56,1.470172e+09


Nossos timestamps estão criados! Podemos droppar todos os atributos inúteis agora:

In [30]:
drop_attribs = [
    'TransactionTime',
    'TransactionDay',
    'TransactionMonth',
    'TransactionYear',
    'TimeHours',
    'TimeMinutes',
    'TimeSeconds'
]

df.drop(columns=drop_attribs, inplace=True)

In [31]:
df.head(5)

,CustGender,CustAccountBalance,TransactionAmount (INR),CustomerAge,TransactionTimestamp
0,1.0,17819.05,25.0,28,1.470159e+09
1,0.0,2270.69,27999.0,65,1.470158e+09
2,1.0,17874.44,459.0,26,1.470159e+09
3,1.0,866503.21,2060.0,49,1.470159e+09
4,1.0,6714.43,1762.5,34,1.470172e+09


### Agrupando

#### MinMaxScaler

In [32]:
from sklearn.preprocessing import MinMaxScaler

data_scaled = MinMaxScaler().fit_transform(df.values)

#### HDBSCAN

In [ ]:
from hdbscan import HDBSCAN

clusterer = HDBSCAN(alpha=1., min_cluster_size=350)
_ = clusterer.fit(data_scaled)

In [ ]:
np.unique(clusterer.labels_, return_counts=True)